# Evaluating the Model

## Introduction

**Learning Objectives**

- Applying the model to make predictions.
- Analysing the results to assess the model's accuracy and effectiveness.

Our model is trained, but is it accurate on new data? We will explore it's prowess on our **test** dataset.

## Setup

To **evaluate** our model we will need to load in both the **validation** dataset and the **test** dataset. The validation data will be used to fine tune our predictions and the test for the actual evaluation

In [ ]:
import numpy as np

from tqdm import tqdm
import pickle

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

### Validation data

In [ ]:
import glob

# Load all chunks
chunks = []
for chunk_file in sorted(glob.glob("./data/embeddings_validation_chunk_*.pkl")):
    with open(chunk_file, 'rb') as file:
        chunks.append(pickle.load(file))

# Combine all chunks
test_dict = {
    'embeddings': [],
    'labels': []
}

for chunk in chunks:
    test_dict['embeddings'].extend(chunk['embeddings'])
    test_dict['labels'].extend(chunk['labels'])

X_valid = np.array(test_dict['embeddings'])
y_valid = test_dict['labels']

In [ ]:
with open("./data/validation_data.pkl", 'rb') as file:
    valid_dict = pickle.load(file)

X_valid = np.array(valid_dict['embeddings'])
y_valid = valid_dict['labels']

### Test data

In [ ]:
# Load all chunks
chunks = []
for chunk_file in sorted(glob.glob("./data/embeddings_test_chunk_*.pkl")):
    with open(chunk_file, 'rb') as file:
        chunks.append(pickle.load(file))

# Combine all chunks
test_dict = {
    'target_id': [],
    'embeddings': [],
    'labels': []
}

for chunk in chunks:
    test_dict['target_id'].extend(chunk['target_id'])
    test_dict['embeddings'].extend(chunk['embeddings'])
    test_dict['labels'].extend(chunk['labels'])

X_test = np.array(test_dict['embeddings'])
y_test = test_dict['labels']

In [ ]:
with open("./data/embs_for_test.pkl", 'rb') as file:
    valid_dict = pickle.load(file)

X_valid = np.array(valid_dict['embeddings'])
y_valid = valid_dict['labels']

We will also need to re-create the model and dataset loader. You can find these below in the collapsed cells.

In [ ]:
class EmbDataset(Dataset): # Subclass Dataset
    def __init__(self, X, y):
        super().__init__()

        self.seq = torch.tensor(X)
        self.label = torch.tensor(y)

    def __len__(self):
        return len(self.seq)

    def __getitem__(self, index):
        return self.seq[index], self.label[index]

In [ ]:
BATCH_SIZE = 32

valid_dataset = EmbDataset(X_valid, y_valid)
test_dataset = EmbDataset(X_test, y_test)

# To dataloader
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)  

In [ ]:
class MLP_LeakyReLu_binary(nn.Module):
    """Simple MLP Model for Classification Tasks."""

    def __init__(self, num_input, num_hidden, num_output=1):
        """
        Initialise the MLP Classifier with input, hidden, and output layers.

        Args:
            num_input (int): The number of input features.
            num_hidden (int): The number of units in the hidden layer.
            num_output (int): The number of output classes.
        """
        # Call the parent class (nn.Module) initializer
        super().__init__()

        # Define the first linear layer (input to hidden)
        self.hidden = nn.Linear(num_input, num_hidden)
        
        # Define the second part of the network (hidden to output)
        # Includes Dropout for regularisation, LeakyReLU activation, and a final linear layer
        self.predict = nn.Sequential(
            nn.Dropout(0.5),            # Apply dropout with 50% probability to prevent overfitting
            nn.LeakyReLU(inplace=True), # Apply LeakyReLU activation with in-place modification
            nn.Linear(num_hidden, num_output) # Fully connected layer for output predictions
        )
        self.sigmoid = nn.Sigmoid()  # Apply sigmoid activation to the output


    def forward(self, x):
        """
        Define the forward calculation process to train the network.

        Args:
            x (torch.Tensor): Input data tensor of shape (batch_size, num_input).

        Returns:
            torch.Tensor: Output predictions of shape (batch_size, num_output).
        """
        x = self.hidden(x)  # Pass the input through the first linear layer (input to hidden)
        x = self.predict(x) # Pass the hidden layer output through the prediction layers
        x = self.sigmoid(x) # Apply sigmoid activation to the output

        return x            # Return the final output

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    # if there are multiple GPUs, choose the first one
    device = torch.device(f"cuda:0")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'Device name: {torch.cuda.get_device_name(0)}')
else:
    print("No GPU detected! Falling back to CPU")
    # If the GPU is not available, use the CPU
    device = torch.device("cpu")

n_input = X_test[0].shape[0]
model = MLP_LeakyReLu_binary(n_input, 1280, 1)

model_dict = torch.load('./data/best_embedding_model.pt', weights_only=True)
model.load_state_dict(model_dict)
model = model.to(device)

## Prediction

Compared to previously we have a lot of test samples to evaluate on, **10 642**. We should therefore build a prediction function that utilises the *batch* data loader. Remember when **predicting** with your model you must put it in **evaluation** mode, otherwise layers such as **dropout** won't be switched off, reducing its capacity.

In [ ]:
def predict(test_loader, model):
    # Put the model in evaluation mode. This disables specific layers such as dropout
    # and batch normalization that behave differently during training and inference.
    model.eval()

    # Initialise empty lists to collect predictions and true labels from each batch.
    preds = []
    labels = []

    # Iterate through each batch in the DataLoader.
    for batch in tqdm(test_loader):

        # Unpack the batch and move the input sequences and labels to the same device
        # as the model to ensure computational alignment.
        b_seq, b_labels = tuple(t.to(device) for t in batch)

        # Disable gradient computation since we are only making predictions (quicker & more efficent)
        with torch.no_grad():

            # Make a forward pass through the model and predict outcomes for the batch.
            pred = model(b_seq.float())
            
            # Append the predicted probabilities to the preds list.
            # with one output 'pred[:, 0]' gets the positive class (binary classification)
            preds.append(pred[:, 0].detach().cpu())
            
            # Append the true labels in the same manner
            labels.append(b_labels.detach().cpu())
        
    # Concatenate all predictions and labels from each batch into a single tensor for each.
    # This makes it easier to compute metrics or further evaluate predictions.
    preds = torch.cat(preds, dim=0)
    labels = torch.cat(labels, dim=0)

    # Return the predictions and true labels for the entire test set.
    return preds, labels

In [ ]:
valid_preds, valid_labels = predict(valid_loader, model)

print(f"\nPrediction: {valid_preds[:5]}\nLabels: {valid_labels[:5]}")

## Classification Threshold

Given that our model outputs probabilities, transforming these into binary predictions (1 or 0) necessitates setting a suitable **threshold**. 

A standard binary threshold is **0.5**. However a more informative  **threshold** can be determined by analysing model performance on the **validation set**. By evaluating various *thresholds'* impact on performance metrics on the **validation set**, you can select the *threshold* that optimises the desired metric—typically balancing sensitivity and specificity or maximising overall accuracy. 

In [ ]:
import pandas as pd

label_map = {0 : 'benign', 1 : 'pathogenic'}

valid_result = pd.DataFrame({'y_proba': valid_preds, 'y_true': valid_labels})
valid_result['labels'] =  valid_result['y_true'].map(label_map)

print(valid_result.head(3))

To determine an optimal threshold for converting model output probabilities into binary class predictions, it’s useful to visualise the distribution of predicted probabilities. The below kernel density plot provides a smoother distribution of the prediction probabilites, seperated by the true label. 

By eye, you should look for a point that best seperates the **majority** of the positive class.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

colour_map = {'benign' : 'green', 'pathogenic' : 'red'}

# Plot the predicted probability density plot
sns.kdeplot(
   data=valid_result, x="y_proba", hue="labels",
   fill=True, common_norm=False, palette=colour_map,
   alpha=.5, linewidth=0,
);
# label
plt.xlabel("Predicted Probability")
plt.ylabel("Density")
plt.title("Predicted Probability Density Plot");

By eye, it appears the boundary that best seperates the two classes is around **0.2**. At the end of this section we will look at some more precise methods to calclate a threshold.

Given this threshold we can now get the predictions for the test datatset, converting them into class predictions.

NOTE BOX

It is imperative any threshold calculations are done on the validation dataset. To use the test dataset would lead to data leakage from unseen data to model.

In [ ]:
# Repeat the above with the test dataset

test_preds, test_labels = predict(test_loader, model)

label_map = {0 : 'benign', 1 : 'pathogenic'}

test_result = pd.DataFrame({'y_proba': test_preds, 'y_true': test_labels})
test_result['labels'] =  test_result['y_true'].map(label_map)

test_result['y_predict'] = test_result['y_proba'] > 0.2

print("\n", test_result.head(3))

## Model metrics

As taught in [**AI2**](../AI2/case-study.html#comparing-classes), two simple methods to assess the efficacy of your model is the **confusion matrix** and the **ROC curve** with the **AUC** score.

For a reminder of that they do click the link back to **AI2**.

However, first lets look at the simple **accuracy** metric to gain an initial understanding of our model's performance.

In [ ]:
from sklearn.metrics import accuracy_score

# Store validation predictions and true labels
y_true = test_result['y_true']
y_pred = test_result['y_predict']
y_pred_proba = test_result['y_proba']

accuracy = accuracy_score(y_true, y_pred)*100
print(f"Accuracy: {accuracy:.2f}%")

With an accuracy of 86% our model is performing well. We can break its performance down by class through the **confusion matrix**.

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(y_true, y_pred, classes=None, normalise=False, colmap=True, ax=None, cmap='inferno'):
    """
    Plot confusion matrix on given axes.
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        classes: List of class labels (if None, uses range from data)
        normalise: If True, normalise the confusion matrix
        ax: Matplotlib axes object (if None, creates new figure)
        cmap: Colourmap for the plot
    """
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Normalise if requested
    if normalise:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Set default classes if not provided
    if classes is None:
        classes = np.unique(y_true)
    
    # Create display
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    
    # Plot on provided axes or create new figure
    disp.plot(ax=ax, cmap=cmap, colorbar=colmap, values_format='.2f' if normalise else 'd')
    
    if ax is not None:
        ax.set_xlabel('Predicted Label')
        ax.set_ylabel('True Label')
    
    return disp

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

plot_confusion_matrix(test_result['y_true'], test_result['y_predict'], 
                         classes=[0,1], 
                         normalise=True,
                         cmap='Blues',
                         ax=ax)
plt.show()

The model....

### Matthews Correlation Coefficient

Matthews Correlation Coefficient (MCC) is a new metric to this module. This coefficient takes into account all four components of the confusion matrix, both positive and negative classes. This holistic approach means that MCC considers the model's performance on both classes equally, making it particularly useful when the classes are imbalanced and when you want to evaluate the model's ability to correctly classify both classes.

The MCC value ranges from **-1 to +1**:
- **+1**: Indicates perfect prediction, the model correctly classifies all samples
- **0**: Suggests the model performs no better than random chance
- **-1**: Indicates complete disagreement between predictions and actual labels (total inverse prediction)

In ptactice a **MCC of 0.5** is considered good, while a **MCC of 0.7** is considered a strong indicator of good performance.

In [ ]:
from sklearn.metrics import matthews_corrcoef

y_true_np = np.array(test_result["y_true"])
preds_bin = np.array(test_result["y_predict"], dtype=int)

MCC = matthews_corrcoef(y_true_np, preds_bin)
print('Test MCC: ', MCC)

### ROC curve

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# plot the ROC curve for each class in the test set as a separate curve on the same plot
fig, ax = plt.subplots(figsize=(8, 8))

fpr, tpr, _ = roc_curve(test_result["y_true"], test_result["y_proba"])
score = roc_auc_score(test_result["y_true"], test_result["y_proba"])
ax.plot(fpr, tpr, label=f"{score:.2f}", c='purple')

ax.plot([0, 1], [0, 1], 'k--', label='Random')

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

## Summary

## Calculate Threshold

This sub-section is a small adendum to this section. Earlier, we chose a threshold of **0.5** by eye. Something that would not hold up to scrutiny in a peer review.

### Top left threshold

A optimal threshold can easily be calculated from the **ROC** curve. Explicitly the threshold that is closest to the top left corner of the 2-D space.

When `roc_curve()` is calculated the true and false positve rates are found a varying thresholds from 0 - 1.

In [ ]:
# Calculate the rates and threshold
fpr, tpr, thresholds_roc = roc_curve(y_true, y_pred_proba)
print(f"There are {len(thresholds_roc)} thresholds") 

From the **FPR** and **TPR** we can caculate the euclidian distance from the top left corner, coordinates - `(0,1)` using the following formula: $$d = \sqrt{FPR^2 + (1 - TPR)^2}$$

In [ ]:
# Find the distances and retrieve the smallest
distances = np.sqrt((fpr - 0)**2 + (tpr - 1)**2)
optimal_idx = np.argmin(distances) # get the smallest distance inde

top_left_threshold = thresholds_roc[optimal_idx]

print(f"The optimal threshold: {top_left_threshold:.2f}")
print(f"with a distance of {distances[optimal_idx]:.3f}")

### Youden's J statistic

[Youden's J statistic](https://en.wikipedia.org/wiki/Youden%27s_J_statistic#Other_metrics) is a related metric to the above. However, rather than select a *threshold* based on the top left distance, it seeks to find the *threshold* that is furthest from the **diagonal line** (random classifer). This can be calculated by minusing the **FPR** from the **TPR**: $$J=TPR−FPR$$

Typically, with a smooth curved **ROC** curve such as the one we obtained, the **Youden's J statistic** will be very similar t the previous method. However, in cases where the curve as a more atypical shape they may diverge. Addtionally, this statistic more explicitly shows a preference to low **False Positives**.

In [ ]:
J = tpr - fpr
optimal_idx = np.argmax(J)

youdens_threshold = thresholds_roc[optimal_idx]

print(f"The youden optimal threshold: {youdens_threshold:.2f}")
print(f"with a distance of {distances[optimal_idx]:.3f}")

PRACTICE EXERCISE

Try running the notebook with a threshold of **0.15**. How does it change the evaluation metrics?